In [ ]:
# 下载数据集
import requests
import os
import re

# 定义数据集的 URL 和校验信息
DATA_URL = 'http://d2l-data.s3-accelerate.amazonaws.com/'  # 假设这是 d2l.DATA_URL 的值
DATA_HUB = {}
DATA_HUB['time_machine'] = (DATA_URL + 'timemachine.txt', '090b5e7e70c295757f55df93cb0a180b9691891a')

def download_time_machine():
    # 使用定义的 URL 来获取文件内容
    url = DATA_HUB['time_machine'][0]
    response = requests.get(url)
    if response.status_code == 200:
        content = response.text
        data_dir = './DataSet'
        if not os.path.exists(data_dir):
            os.makedirs(data_dir)
        file_path = os.path.join(data_dir, 'timemachine.txt')
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content)
        print('文件下载成功！')
    else:
        print('文件下载失败！')
download_time_machine()


# 验证文件
try:
    with open("./DataSet/timemachine.txt", "r", encoding="utf-8") as file:
        text = file.read()
    print("文件读取成功")
except FileNotFoundError:
    print("文件未找到")
except UnicodeDecodeError:
    print("文件编码错误")

In [ ]:
# 读取数据集
def read_time_machine():
    """将《时间机器》数据集加载到文本行列表中。"""
    with open('./DataSet/timemachine.txt', 'r', encoding='utf-8') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]


lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

In [ ]:
# 分词词元化
# 下面的`tokenize`函数将文本行列表（`lines`）作为输入，
# 列表中的每个元素是一个文本序列（如一条文本行）。
# [**每个文本序列又被拆分成一个词元列表**]，*词元*（token）是文本的基本单位。
# 最后，返回一个由词元列表组成的列表，其中的每个词元都是一个字符串（string）。
def tokenize(lines, token='word'):  # @save
    """将文本行拆分为单词或字符词元"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型：' + token)
#检测 tokenize
tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

In [ ]:
# 构建词汇表
# 词元的类型是字符串，而模型需要的输入是数字，因此这种类型不方便模型使用。
# 现在，让我们[**构建一个字典，通常也叫做*词表*（vocabulary），
# 用来将字符串类型的词元映射到从$0$开始的数字索引中**]。
# 我们先将训练集中的所有文档合并在一起，对它们的唯一词元进行统计，
# 得到的统计结果称之为*语料*（corpus）。
# 然后根据每个唯一词元的出现频率，为其分配一个数字索引。
# 很少出现的词元通常被移除，这可以降低复杂性。
# 另外，语料库中不存在或已删除的任何词元都将映射到一个特定的未知词元“&lt;unk&gt;”。
# 我们可以选择增加一个列表，用于保存那些被保留的词元，
# 例如：填充词元（“&lt;pad&gt;”）；
# 序列开始词元（“&lt;bos&gt;”）；
# 序列结束词元（“&lt;eos&gt;”）。
import collections


class Vocab:  # @save
    """文本词表"""

    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现频率排序
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                   reverse=True)
        # 未知词元的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs


def count_corpus(tokens):  # @save
    """统计词元的频率"""
    # 这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)


# 加载语料库
def load_corpus_time_machine(max_tokens=-1):
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

In [ ]:
#用数据集作为语料库构建词表，然后打印前10个高频词元及其索引
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])